# OpenAI API를 이용하여 뉴스 기사 정리 봇 만들기

OpenAI Chat API를 이용해 뉴스 검색 결과를 정리하는 프로그램을 만들어보기


크롤링 전용 라이브러리가 필요하지만, 이번 실습에서는 네이버의 뉴스 검색 API를 사용.

In [ ]:
!pip install openai tiktoken --upgrade

In [ ]:
#key 입력

## 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기

네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성하겠습니다.   
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 {"title", "URL", "desc"} 의 dictionary 형식으로 파싱하고,   
이를 리스트로 만들어 json으로 return하는 함수 get_html(query) 를 만들고 싶습니다.   <br>
   
위 요청을 그대로 전달하면, 함수를 만들 수 있을 것 같습니다.


**네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 올라갑니다.**

In [ ]:
prompt_instuction = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 
{"title", "URL", "desc"} 의 dictionary 형식으로 파싱하고,
이를 리스트로 만들어 json으로 return하는 함수 get_html(query) 를 만들면 됩니다.
---
'''

prompt = '''

# 검색어
query = '생성형 AI'


# 검색 결과를 return하는 함수
def get_html(query):


'''



In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{'role':'system', 'content':'당신은 파이썬 코딩의 전문가입니다.'},
          {'role':'user', 'content':prompt_instuction + prompt}],

    temperature =  0.1,
    max_tokens = 1000

)
print(response.choices[0].message.content)

In [ ]:
# VS CODE에서 위 결과가 truncated 되는 경우 scrollable element를 클릭하여 풀 코드 확인

아래에 코드를 넣고 실행해 보겠습니다.

단, API 키는 LLM으로 가져올 수 없으므로 아래의 키를 사용합니다.

In [ ]:
headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': ''
}

아래는 모범 답안입니다. 결과가 조금 다르게 나오더라도 이후 코드를 수정하면 되므로 괜찮습니다.

In [ ]:
# # 전체 코드를 선택 후 Ctrl+/ 로 일괄 주석해제
# import requests
# import json

# def get_html(query):
#     # 네이버 API URL
#     url = "https://openapi.naver.com/v1/search/news.json"

#     # API 요청 헤더
#     headers = {
#     'X-Naver-Client-Id': '',
#     'X-Naver-Client-Secret': ''

#     }

#     # API 요청 파라미터
#     params = {
#         "query": query,
#         "display": 30,  # 검색 결과 개수
#         "sort": "sim"   # 관련도순 정렬
#     }

#     # API 요청
#     response = requests.get(url, headers=headers, params=params)

#     # 응답이 성공적일 경우
#     if response.status_code == 200:
#         data = response.json()
#         articles = data.get('items', [])

#         # 결과를 저장할 리스트
#         result = []

#         # 기사 정보 파싱
#         for article in articles:
#             result.append({
#                 "title": article['title'],
#                 "URL": article['link'],
#                 "desc": article['description']
#             })

#         # JSON 형식으로 반환
#         return json.dumps(result, ensure_ascii=False)

#     else:
#         return json.dumps({"error": "Failed to fetch data"}, ensure_ascii=False)

# # 검색어
# query = '생성형 AI'

# # 함수 호출
# result = get_html(query)
# print(result)

## 2. OpenAI chat을 이용하여 요약하기

키워드에 대한 뉴스 검색 결과를 요약합니다.   
위 결과인 news_json 데이터를 입력으로 넣은 뒤, 해당 결과에 대한 요약을 요청합니다.

In [ ]:
def news_bot(messages):

    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens= 2048,
    )

    return result.choices[0].message.content

챗봇에 전달할 메시지는 두 개로 나눠집니다.   
위에서 얻은 get_html(query)와, 사용자가 제시할 명령입니다.   

In [ ]:
query = '삼성 라이온즈'

news_json = get_html(query)

result = news_bot([
    {
        "role": "system",
        "content": query + """에 대한 뉴스 검색 결과가 주어집니다. """+ query+"""에 대한 뉴스를
요약하세요. 주제별로 링크를 포함하세요.
---
"""
    },
    {
        "role": "user",
        "content": get_html(query)
    }
])

print(result)

In [ ]:
# 토큰 수 체크

import tiktoken
tokenizer  = tiktoken.encoding_for_model('gpt-4o-mini')

print("문자 수:", len(news_json))
print("토큰 수:", len(tokenizer.encode(news_json)))
news_json


다양한 명령을 넣어서 형식과 스타일을 바꿀 수 있습니다.

# 실습 query의 뉴스 내용을 기반으로 서로 대화하는 형식의 답변 출력하기

In [ ]:
query = ''

news_json  = get_html(query)

result = news_bot([
    {
        "role": "user",
        "content": news_json
    },
    {
        "role": "user",
        "content": f"""
내용 입력
        """
    }
])

print(result)

## 참고) 스트리밍
ChatGPT처럼 글자가 한 글자씩 순서대로 쭉 출력되도록 하면 어떨까요?   
이와 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있습니다.

In [ ]:
def news_bot_stream(messages):

    result = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens = 2048,
        stream=True # 스트리밍
    )

    return result

In [ ]:
query = '엔비디아 주가'
news_json = get_html(query)

result = news_bot_stream([
    {
        "role": "user",
        "content": news_json
    },
    {
        "role": "user",
        "content": """
위 전체 내용을 종합하여 친구에게 전달하는 편지를 작성해줘. 친절하고 상냥한 말투를 사용하고, 형식은 아래와 같아.
---
'제목':
'본문':
        """
    }
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

이와 같은 스트리밍은 LLM 어플리케이션의 마지막 출력에서 사용하면    
사용자 경험을 향상시킬 수 있습니다.

## 심화) 챗봇 기능 고도화하기

위에서 만든 news_bot에, 관련없는 질문을 입력하면 어떻게 될까요?

In [ ]:
news_json

In [ ]:
result = news_bot([
    {
        "role": "user",
        "content": news_json
    },
    {
        "role": "user",
        "content": "오늘 저녁 뭐 먹을까? 타코와 피자 중에서."
    },

])

print(result)

그럭저럭 대답을 잘 하고 있습니다만, query를 통해 검색한 결과와는 관련이 없습니다.   
만약 특정 태스크만을 수행하는 챗봇을 만들었다면, 이와 같은 응답은 **환각(Hallucination)** 으로 간주될 수 있으며,   
이와 같은 행동을 쉽게 허용한다면 예상치 못한 출력을 생성할지도 모릅니다.


환각을 방지하기 위해, 프롬프트를 조금 더 강화시켜 보겠습니다.    
챗봇의 메시지 중 `system`을 이용하면, 챗봇의 행동을 더 명시적으로 전달할 수 있습니다.

## 실습) 튼튼한(?) 챗봇 만들기
위와 같이 주어진 정보와 관련없는 데이터가 주어질 경우, 이를 방어할 수 있는 챗봇 프롬프트를 구성하세요.     
예를 들어, news_json과 관련이 없는 질문이 나오면 답변을 거절하면 될 것 같습니다.

In [ ]:
query = 'LLM'
news_json = get_html(query)
news_json

In [ ]:
result = news_bot([
    {'role':'system',
     'content':'''
내용 입력
'''
     },
    {
        "role": "user",
        "content": '[정보]: ' + news_json
    },
    {
        "role": "user",
        "content": "[질문]: "
    },

])

print(result)

이번 실습 시간에 만들어 본 봇의 기능을 더 추가하는 방법에는 무엇이 있을까요?
<br><br>
- 검색 결과에서 최신 기사만 필터링하기
- query 검색 결과에서 특정 단어가 포함된 기사는 제외하기
- 각 기사들의 설명을 보고 해당 단어에 대한 긍정(부정)적인 기사만 사용하여 요약하기    

이와 같은 방법들은 실제로 사용되는 방식이므로, 이를 활용하면 챗봇의 기능을 더 고도화할 수 있습니다.

## 부록) Prompt Injection

적대적 프롬프트(Adversarial Prompt)를 통해 챗봇의 기능을 무력화하고, 정보를 유출시키는 프롬프트도 있습니다.

https://www.promptingguide.ai/kr/risks/adversarial    
https://github.com/0xk1h0/ChatGPT_DAN    
